# <center> Travaux pratiques d’IA
## <center> Série 4: Decision Trees
#### <center> Michel Donnet

## Données
Les données à utiliser pour ce TP se trouvent dans les fichiers data.csv et data_test.csv.
Les 5 premières colonnes spécifient les variables indépendantes tandis que la dernière colonne correspond à la variable dépendante (label).
Le fichier data_test.csv sert à évaluer les performances des arbres développés à partir des données du fichier data.csv.

J'ai défini la fonction suivante afin de convertir un fichier csv en np.array:

In [4]:
import csv
import numpy as np
import random


def convert_csv2array(name: str) -> tuple[np.ndarray, np.ndarray]:
    """
    Convert a csv file into numpy array

    Parameters
    ----------
    name: string
        Path to the file

    Return value
    ------------
    header and data of csv
    """
    file = open(name, 'r')
    data = []
    reader = csv.reader(file)
    for line in reader:
        data.append(line)
    data = np.array(data)
    return data[0, :], data[1:, :].astype(int)

header, data = convert_csv2array('data.csv')

# Exercice 1: Entropie et gain d’information

1. Calculer l’entropie de la variable dépendante.

In [5]:
def entropy(data: np.ndarray, column: int) -> float:
    """
    Give the entropy of the given column of data

    Parameters
    ----------
    data: np.ndarray
        Data
    column: int
        column on which we want to compute entropy

    Return value
    ------------
    The entropy of the given column of data 
    """
    values, count = np.unique(data[:, column], return_counts=True)
    result = 0
    for i in range(len(values)):
        nb = count[i] / len(data)
        result += -nb * np.log2(nb)
    return result

print(f"Entropy of the independant variable: {entropy(data, 5)}")

Entropy of the independant variable: 0.9738003694382252


2. Calculer le gain d’information réalisé après l’application de trois critères de décision aléatoires.

In [8]:
def conditionnal_entropy(data: np.ndarray, column_1: int, column_2: int) -> float:
    """
    Compute conditionnal entropy H(X|Y)

    Parameters
    ----------
    data: np.ndarray
        Data used to compute conditionnal entropy
    column_1: int
        Column of X
    column_2: int
        Column of Y

    Return value
    ------------
    Return the result of H(X|Y)
    """
    values, count = np.unique(data[:, column_2], return_counts=True)
    result = 0
    for i in range(len(values)):
        newdata = data[data[:, column_2] == values[i]]
        nb = count[i] / len(data)
        result += nb * entropy(newdata, column_1)
    return result

def mutual_information(data: np.ndarray, column_1: int, column_2: int) -> float:
    """
    Compute mutual information I(X; Y)
    
    The gain in information is represented by the mutual information between 2 random variables

    Parameters
    ----------
    data: np.ndarray
        Data used to compute mutual information
    column_1: int
        column of X
    column_2: int
        column of Y

    Return value
    ------------
    Return the result of I(X; Y) = H(X) - H(X|Y)
    """
    return entropy(data, column_1) - conditionnal_entropy(data, column_1, column_2)

randnumber_1 = random.randint(0, 4)
randnumber_2 = random.randint(0, 4)
randnumber_3 = random.randint(0, 4)
while randnumber_1 == randnumber_2:
    randnumber_2 = random.randint(0, 4)
while randnumber_3 == randnumber_1 or randnumber_3 == randnumber_2:
    randnumber_3 = random.randint(0, 4)
print(f"For the criteria {header[randnumber_1]}, the gain of information is: {mutual_information(data, 5, randnumber_1)}")
print(f"For the criteria {header[randnumber_2]}, the gain of information is: {mutual_information(data, 5, randnumber_2)}")
print(f"For the criteria {header[randnumber_3]}, the gain of information is: {mutual_information(data, 5, randnumber_3)}")

For the criteria A, the gain of information is: 0.02894167191284036
For the criteria D, the gain of information is: 0.16681614778733878
For the criteria C, the gain of information is: 0.013270151845539147


3. Pour les mêmes critères de décision, calculer l’index Gini.

In [10]:
def gini(data: np.ndarray, column: int) -> float:
    """
    Compute the gini index

    Parameters
    ----------
    data: np.ndarray
        Data used to compute gini index
    column: int
        column used to compute gini index

    Return value
    ------------
    Return the index gini
    """
    values, count = np.unique(data[:, column], return_counts=True)
    result = 0
    for i in range(len(values)):
        nb = count[i] / len(data)
        result += nb * nb
    return 1 - result

print(f"For the criteria {header[randnumber_1]}, the gini index is: {gini(data, randnumber_1)}")
print(f"For the criteria {header[randnumber_2]}, the gini index is: {gini(data, randnumber_2)}")
print(f"For the criteria {header[randnumber_3]}, the gini index is: {gini(data, randnumber_3)}")

For the criteria A, the gini index is: 0.5733999999999999
For the criteria D, the gini index is: 0.7758499999999999
For the criteria C, the gini index is: 0.67795


4. Quel est le critère de décision préférable selon le gain d’information ? Selon l’index Gini ?

# Exercice 2: ID3

1. Implémenter l’algorithme ID3 avec comme critères possibles le gain d’information et l’index Gini.

2. Comparer l’arbre obtenu à l’aide d’ID3 gain d’information avec celui produit par la démonstration.

3. Implémenter une procédure de génération de données à partir d’un arbre de décision.

4. À l’aide d’ID3 gain d’information, construire 5 arbres à partir d’échantillons aléatoires de 80% des données et utiliser comme prédiction finale un vote de majorité.

5. Comparer les performances du premier arbre obtenu avec celles de l’ensemble de 5 arbres selon les métriques suivantes: accuracy, precision, recall, F1 score.

6. Selon le F1 score, quel modèle devrait être privilégié ?